## ANIMAL CLASSIFICATION - ASSIGNMENT 2

In previous assignment, we solved a binary classification problem for tabular data by using a simple Multilayer Perceptron (MLP). The aim was to familiarise yourself with some basic commands to implement a Deep Learning (DL) model and to emphasise the importance of knowing your data. We are now ready to move on to the next level!

Here, we will work with Convolutional Neural Networks (CNNs). As you have learn during this course, there are some families of DL architectures that are more suitable for a particular task depending on the problem to be solved, the data you are working with, etc. Introduced by Yann LeCun in 1989, CNNs have been the most used architectures when it comes to Computer Vision applications like Image Classification (until the Transformers came along, but that is a different topic...). 

In this assignment, we aim to implement a CNN in order to *classify images of 10 different animals*. One could wonder, is classifying animals that useful? And the answer is.. probably not! But think about the big picture: you could learn to classify (and diagnose) a specific disease based on medical images, for instance. We will split this assignment in 4 sections:

* EDA and Data Preprocessing
* CNN Implementation
* Data Augmentation
* Transfer Learning

As we did before, the first step is to understand our data. As we are working with images, we will focus on gaining some knowledge about the distribution of our data and how to deal with it, as well as displaying some image examples. Then, we will prepare the data to be fed in our CNN. Then, we will implement a basic Convolutional Neural Network to train our multiclass classifier since we deal with images as our input. We will also apply some data augmentation to see the influence of this technique during training. Finally, we will make use of a pre-existing model from the Keras library - namely VGG16, one of the very first "well-known" convolution-based architectures. We will experiment with what is called *Transfer learning*, a really common practice in the DL world in which we simply make use of pre-existing weight values (features) for our architecture. Those pre-existing values were obtained by training the architecture on a large corpus of images (ImageNet) and can be used to accelerate the training process.

Let's start!

**NOTE**: Our model deployment will be carried out with **TensorFlow**. In Kaggle, we can make use of some free GPUs available to speed up the training process. To run notebooks with GPU (we will really need it in this assignment), select the GPU P100 option in the accelerator setting. You will find this option by clicking in the three vertical dots in the top-right corner of the Notebook.

**NOTE**: In Keras, there are two ways of defining a model: Sequential (as previous assignment) and Functional API. In this assignment, we are going to use the Keras Functional API. To get familiar with this flexible way to create models, please take a quick look to the Introduction section in [this tutorial](https://www.tensorflow.org/guide/keras/functional#introduction).

**NOTE**: Througout the different tasks in this assignment, you will find some questions marked as **Q**. These questions should be answered at the end of the Notebook (there is a Markdown cell prepared for this purpose).

### Installing Libraries

To enable the installation of libraries in Kaggle, go to **Notebook options** and activate *Internet* option. Then, install the following libraries:

In [ ]:
!pip install keras==2.12.0
!pip install matplotlib==3.6.3
!pip install numpy==1.23.5
!pip install pandas==1.5.3
!pip install scikit_learn==1.2.2
!pip install seaborn==0.12.2
!pip install tensorflow==2.12.0

### EDA and Data Preprocessing

The first thing you'll need to do, is to load the dataset from Kaggle. To do that, we need to add the dataset to our notebook. This can be done by clicking in *Add data* in the top-right corner of the notebook and searching for **Animals-10** (Corrado Alessio). Once the dataset has been added (it should appear in our notebook's input, again in the top-right corner), we can start. If you want to learn more about the information contained in this dataset, check https://www.kaggle.com/datasets/alessiocorrado99/animals10.

In [ ]:
# Imports
import pandas as pd

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import os
from pathlib import PurePosixPath as PPP

from sklearn.preprocessing import LabelEncoder
import seaborn as sns

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

**Task 1**

As you can see, our data is stored by classes. Our first task is to read the dataset and create the independent variable containing our ground truth (labels) for the classification task. Once we have created our array containing the values of the ground truth, we can store them in a dataframe (we are already familiar with Pandas). Store the name of the file and its associated category in a dataframe.

In [ ]:
# Read all files
raw_path = "../input/animals10/raw-img"
filename_data = []
for (dirpath, dirnames, filenames) in os.walk(raw_path):
    if '.py' in filenames: continue # Skip python file included in the main folder
    filename_data  += [str(PPP(dirpath).joinpath(file)) for file in filenames]
    
# Ground truth
categories = []

# Loop over the filenames and use "split" to get the category name.
for file in filename_data:
    categories.append() # Type your solution here
        
# Create a dataframe containing the names of the files and the categories. Use 'filename' and 'labels' as column names.
df =  # Type your solution here.

# Show random samples
df.sample(n=9, random_state=6)

As you can see, the original dataset is in Italian. Let's make use of the dictionary contained in *translate.py* to translate the labels. It will be much easier to work in English :)

In [ ]:
# Dictionary
label_mapping = {"cane": "dog", "cavallo": "horse", "elefante": "elephant", "farfalla": "butterfly", "gallina": "chicken", 
                 "gatto": "cat", "mucca": "cow", "pecora": "sheep", "scoiattolo": "squirrel","ragno": "spyder"}

# Use .map() function to translate the 'labels' column
df['labels'] =  # Type your solution here.

# Show same random samples
df.sample(n=9, random_state=6)

**Task 2**

Use the "labels" of the dataframe and value_counts() to plot a bar plot showing the class distribution of the data.

**Q1**: As you can see, the data is not equally distributed. An alternative could be to resampling our dataset (oversampling or undersampling) until the dataset is balanced. In our case, we will not consider this but, can you think of ways to solved the imbalanced data issue? Do you know any particular application where data is usually highly unbalanced?

In [ ]:
# Get the value counts for each label
label_counts =  # Type your solution here.

# Create the figure and axes
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20, 6))

# Plot the bar chart using label_counts index and values
sns.barplot(x=, y=, alpha=0.8, palette='pastel', ax=axes) # Type your solution here.
axes.set_title('Distribution of Labels in Image Dataset', fontsize=16)
axes.set_xlabel('Label', fontsize=14)
axes.set_ylabel('Count', fontsize=14)
axes.set_xticklabels(label_counts.index, rotation=45)

# Add a super-title to the figure
fig.suptitle('Image Dataset Label Distribution', fontsize=20)

# Adjust the spacing between the plots and the title
fig.subplots_adjust(top=0.85)

# Display the plot
plt.show()

print("Number of images per category : ")
print(label_counts)

**Task 3** 

Once we have loaded the dataset, we will proceed to show some examples. Plot the same random samples we have been showing before to get an idea of how our data looks like.

In [ ]:
# Extract 9 random samples using random_state=6. Remember to reset indexes with reset_inex()
sample_df =  # Type your solution here.

plt.figure(figsize=(12, 12))
for index, row in sample_df.iterrows():
    filename =  # Type your solution here.
    category =  # Type your solution here.
    img = load_img(filename, target_size=(256, 256))
    plt.subplot(3, 3, index+1)
    plt.imshow(img)
    plt.xlabel(category)
plt.tight_layout()
plt.show()

**Task 4**

Similar to Assignment 1, we need to convert the non-numerical variables (categorical) to numerical. In this case, we will make use of the LabelEncoder() function to encoder our labels and store them in a new "encoded_labels" column.

In [ ]:
# Create a encoder object
lb =  # Type your solution here.

# Use fit_transform to fit the data and encode it
df['encoded_labels'] =  # Type your solution here.

# Show same random samples
df.sample(n=9, random_state=6)

**Task 5** 

Time to prepare the data for the architecture! Remember that we already have the dataframe containing the names and class of each image. We will split in training, validation and test. Make use of the dataframe to obtain a train set with 70% of the data. The remaining data will be equally divided into validation and test set. Use train_test_split() funtion including *suffle=True* , *random_state=0* and *stratify* arguments when splitting. More info here https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
# Split dataset
train_df, Temp_df = train_test_split(, train_size=, shuffle=True, 
                                     random_state=0, stratify=df['labels'])  # Type your solution here
valid_df, test_df = train_test_split(, train_size=, shuffle=True, 
                                     random_state=0, stratify=Temp_df['labels']) # Type your solution here

train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print("Shape of the training set:", train_df.shape)
print("Shape of the validation set:", valid_df.shape)
print("Shape of the testing set:", test_df.shape)

**Task 6**

For feeding the data into our model, we will make use of the ImageDataGenerator() class. Here, we can use some of the predefined transforming/preprocessing methods included. Use flow_from_dataframe() with an ImageDataGenerator that will only apply a rescale to the images (1./255) with a batch size of 64. We will also resize our images to a (224,224). Later, we will use such generator with more operations to perform data augmentation at training time but for now, let's keep it simple. The reescaling operation simply makes sure that all the pixel values of the images fall in between a defined range [0-1], which helps the training of the network.

In [ ]:
# Convert labels to string
train_df["encoded_labels"] = train_df['encoded_labels'].astype(str)
valid_df["encoded_labels"] = valid_df['encoded_labels'].astype(str)

train_datagen = ImageDataGenerator(
    rescale=, # Type your solution here
)

# Include the column name for x and y
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df, 
    x_col=, # Type your solution here
    y_col=, # Type your solution here
    class_mode='categorical',
    target_size=, # Type your solution here
    batch_size= 64
)

validation_datagen = ImageDataGenerator(
    rescale=, # Type your solution here
)

# Include the column name for x and y
validation_generator = validation_datagen.flow_from_dataframe(
    dataframe=valid_df, 
    x_col=, # Type your solution here
    y_col=, # Type your solution here
    class_mode='categorical',
    target_size=, # Type your solution here
    batch_size= 64
)

### CNN Implementation

Time to implement our classifier! We will first create our "homemade" model. Later on, we will make use of one of the well-known architectures instead. Unlike assignment 1, dealing with images is computacionally heavy so the trainining process is slower. Please, make sure you followed the steps mentioned in the introduction of this Notebook to use of the Kaggle's GPUs. We will run just a few epochs (around 10 min), but take into account that, in real applications, training can last for thousands of epochs depending on the task.

**Task 7** 

Create a 4-layer CNN with 32, 64, 128 and 128 units respectively with a (3,3) kernel and Rectified Linear Unit as activation function. Add a MaxPooling2D after each CNN layer with pool size (2,2) to reduce the spatial dimension.

**Q2**: Two of the important ideas that CNNs leverages are *sparse interaction* (or locally connected layers) and *parameter sharing*. Explain briefly both of them.

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, GlobalMaxPooling2D, Input
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model

# Which input shape? Remember to include the number of channels...
input_shape = () # Type your solution here.

# How many classes?
num_classes =  # Type your solution here.

def create_basic_model(input_shape):
    
    # Use the Input layer with the right input shape
    inputs = Input(shape=) # Type your solution here
    
    # Basic CNN model. Type your solution here. 
    x = 
    x = 
    x = 
    x = 
    x = 
    x = 
    x = 
    x = 
    
    # Add a Dropout with prob. 0.5
    x =  # Type your solution here
    
    # Flatten the output layer to 1 dimension.
    x =  # Type your solution here
    
    # Add a fully connected layer with 512 hidden units and ReLU activation.
    x =  # Type your solution here
    
    # Add the final layer (Dense) with the right activation.
    output =  # Type your solution here
    
    # Create a Model() with the right input and output
    model =  # Type your solution here
    
    # Show a summary of the model to make sure there are no disconnected elements.
    # Type your solution here
    
    return model

model = create_basic_model(input_shape)

**Task 8**

Let's train the model! Use fit() with the generators that we have already defined (validation and training). Run it for 10 epochs with Adam optimizer and learning rate of 0.001. Use the total amount of samples (obtained from the dataframe) to define the number of steps for both training (steps_per_epoch) and validation (validation_steps) (total_samples//batch_size). Plot the curves (loss and accuracy).

In [ ]:
#  Use compile with the right loss, optimizer and metrics
model.compile(loss=, # Type your solution here.
              optimizer=, # Type your solution here.
              metrics=) # Type your solution here.

# Training
history = model.fit(
    x=, # Type your solution here (use the training generator),
    epochs=, # Type your solution here
    validation_data=, # type your solution here (use validation generator),
    validation_steps=, # Type your solution here,
    steps_per_epoch=, # Type your solution here
)

In [ ]:
# Plot the accuracy curves
f,ax=plt.subplots(1,2,figsize=(18,8))

ax[0].plot() # Type your solution
ax[0].plot() # Type your solution
ax[0].set_title('model accuracy')
ax[0].set_ylabel('accuracy')
ax[0].set_xlabel('epoch')
ax[0].legend(['train', 'val'], loc='upper left')

ax[1].plot() # Type your solution
ax[1].plot() # Type your solution
ax[1].set_title('model loss')
ax[1].set_ylabel('loss')
ax[1].set_xlabel('epoch')
ax[1].legend(['train', 'val'], loc='upper left')

### Data Augmentation

As you've seen, training a convolutional neural network can be quite a long process, even when resources such as GPU are available. Moreover, the progress can be quite slow in terms of improved performance over time - our results in the previous run were not particularly impressive! As we see in the plots, the validation dataset seems to flatten over time in performance, whereas the training is almost perfect. This is an indicator of overfitting in our model. Nevertheless, there are some techniques than can help with this situation. Data augmentation refers to applying transformations during training such that the network learns that a cat is still a cat even if it, for instance, rotated!. Augmentation can be particularly useful when dealing with small datasets, as it is a way to "have more information" without the burden of labeling more data.

**SPOILER**: The training process is going to be even slower than before...

**Task 9**

Re-define our data generators but this time apply rotation (30 degrees), horizontal flip, width and height shift (0.1) and zooming (0.2). Be careful, since we want to apply the augmentations to the training set ONLY. Display the same sample generated by the new generators after applying the different transformations.

In [ ]:
# Train generator with Data Augmentation
train_datagen_aug = ImageDataGenerator(
        rescale=1./255,
        rotation_range=, # Type your solution here
        width_shift_range=, # Type your solution here
        height_shift_range=, # Type your solution here
        zoom_range=, # Type your solution here
        horizontal_flip=, # Type your solution here
        fill_mode='nearest'
)

train_generator_aug = train_datagen_aug.flow_from_dataframe(
    dataframe=train_df, 
    x_col=, # Type your solution here
    y_col=, # Type your solution here
    class_mode=, # Type your solution here
    target_size=(224, 224),
    batch_size= 64
)

In [ ]:
# Extact only one sample from the training et
sample_df_aug =  # Type your solution here

# Create an example_generator using the train_datagen_aug
example_generator = train_datagen_aug.flow_from_dataframe(
    dataframe=sample_df_aug,  # Type your solution here
    x_col=, # Type your solution here
    y_col=, # Type your solution here
    class_mode=, # Type your solution here
) # Type your solution here

plt.figure(figsize=(12, 12))

for i in range(0, 9):
    plt.subplot(3, 3, i+1)
    for X_batch, Y_batch in example_generator:
        image = X_batch[0]
        plt.imshow(image)
        break
plt.tight_layout()
plt.show()

**Task 10** 

Let's train a new model with the new generators for 10 epochs. Compile the model and train it with the same configuration previously used. Plot the new validation/train curves.

In [ ]:
# Create a new model by calling to create_basic_model().
model_aug =  # Type your solution here.

# Use compile with the right loss, optimizer and metrics
model_aug.compile(loss=, # Type your solution here.
              optimizer=, # Type your solution here.
              metrics=) # Type your solution here.

# Training
history_aug = model_aug.fit(
    x=, # Type your solution here (use the training generator),
    epochs=, # Type your solution here
    validation_data=, # type your solution here (use validation generator),
    validation_steps=, # Type your solution here,
    steps_per_epoch=, # Type your solution here
)

In [ ]:
# Plot the accuracy curves
f,ax=plt.subplots(1,2,figsize=(18,8))

ax[0].plot() # Type your solution
ax[0].plot() # Type your solution
ax[0].set_title('model accuracy')
ax[0].set_ylabel('accuracy')
ax[0].set_xlabel('epoch')
ax[0].legend(['train', 'val'], loc='upper left')

ax[1].plot() # Type your solution
ax[1].plot() # Type your solution
ax[1].set_title('model loss')
ax[1].set_ylabel('loss')
ax[1].set_xlabel('epoch')
ax[1].legend(['train', 'val'], loc='upper left')

### Transfer Learning

As you've verified, data augmentation may offer improvement over the basic convolutional setting. In this case, with the same amount of epochs, the validation performance is similar than the previous experiment, but there is still room for improving on the training set (and, hopefully, on the validation set as well). Although the more preprocessing steps we add to our generator the slower the training process is, this technique usually helps the model to generalize better. Let's try a different thing... Like it has been mentioned, transfer learning is a really common approach to DL problems. A lot of tasks/datasets benefit from using ImageNet pretrained weights. Let's try to use them and see the effect on our task! 

**NOTE**: In this experiment, we skip the use of data augmentation to train our model faster.

**Task 11** 

Create a model that makes use of ImageNet pretraining weights. Make use of keras.applications VGG16. Don't include the classification head (top) of VGG16, we will add our own custom one instead. USe GlobalMaxPooling2D to flatten the output of VGG16 and then add a Dense layer with 512 hidden units and ReLU activation. Following, add a Dropout layer with 0.5 probability. Finally, add a Dense layer with the right number of units and activation function. Finally, we will freeze the whole architecture excet for the last layers (7 onwards). Feel free to experiment with the number of frozen layers to see the effect of fine-tuning different layers.

In [ ]:
from tensorflow.keras.applications import VGG16

# Which input shape? Remember to include the number of channels...
input_shape = () # Type your solution here.

# How many classes?
num_classes =  # Type your solution here.

# Number of trainable layers
train_layer =  # Type your solution here.

def create_ImageNet_model(input_shape, TRAINABLE_LAYERS):

    vgg16 = VGG16(input_shape=, include_top=, weights="imagenet") # Type your solution here

    # Flatten the VGG16 output layer to 1 dimension with GlobalMaxPooling2D()
    x = # Type your solution here
    
    # Add a fully connected layer with 512 hidden units and ReLU activation
    x = # Type your solution here
    
    # Add a dropout rate of 0.5
    x = # Type your solution here
    
    # Add a final layer to classify
    output = # Type your solution here

    # Create a Model() with the right input and output.
    model = # Type your solution here
    
    # Loop over the layers that we want to set as trainable (True).
    # Leave the rest of layers frozen (False)
    for layer in model.layers[:-TRAINABLE_LAYERS]:
        layer.trainable = 
    
    for layer in model.layers[-TRAINABLE_LAYERS:]: # Type your solution here
        layer.trainable =  # Type your solution here

    # Print a summary of the model to make sure there are no disconnected elements.
    # Type your solution here
    
    return model

# Create model
model_imgnet = create_ImageNet_model(input_shape, train_layer)

**Task 12** 

Transfer learning has not only benefits in terms of final performance but also in terms of training speed and convergence - it is the right moment to introduce an early stopping mechanism!. Such approach pre-defines a criteria to stop the training if certain conditions are met. For intstance, if the validation loss does not improve for 5 epochs -> training is stopped. Implement the early stopping callback and monitorize the validation loss. Stop the training if there is no improvement for more than 3 consecutive epochs. You should take into account that the point in which the training has been stopped is probably not the optimal one in terms of performance. That is why, usually, we make use of another callback in addition to early stopping. Create a model checkpoint callback. We will recover (load the weights) of the best epoch in terms of performance after the early stopping has been triggered. The checkpoint should focus on maximum model accuracy. Save the best weights only.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

# Type your solution here (choose the right values)
es = EarlyStopping(monitor=, mode="auto", verbose=1, patience=)

# Type your solution here
mc = ModelCheckpoint('best_model.h5', monitor=, mode=, verbose=1, save_best_only=True)

**Task 13** 

Train the VGG16 for 10 epochs. In this case, VGG16 is known to work better with SGD as optimizer. We'll use a learning rate of 1e-4 and a momentum of 0.9. Plot the new validation/train curves.

In [ ]:
# Use compile with the right loss, optimizer and metrics
model_imgnet.compile(loss=, # Type your solution here
              optimizer=, # Type your solution here
              metrics=) # Type your solution here

# Training
history_imgnet = model_imgnet.fit(
    x=, # Type your solution here (use the training generator without augmentation),
    epochs=, # Type your solution here
    validation_data=, # Type your solution here (use validation generator)
    validation_steps=, # Type your solution here
    steps_per_epoch=, # Type your solution here
    callbacks=) # Type your solution here

In [ ]:
# Plot the accuracy curves
f,ax=plt.subplots(1,2,figsize=(18,8))

ax[0].plot() # Type your solution
ax[0].plot() # Type your solution
ax[0].set_title('model accuracy')
ax[0].set_ylabel('accuracy')
ax[0].set_xlabel('epoch')
ax[0].legend(['train', 'val'], loc='upper left')

ax[1].plot() # Type your solution
ax[1].plot() # Type your solution
ax[1].set_title('model loss')
ax[1].set_ylabel('loss')
ax[1].set_xlabel('epoch')
ax[1].legend(['train', 'val'], loc='upper left')

**Task 14**

We see that we can benefit a lot of pretrained models. With just a few epochs, we have been able to outperform previous models with a basic CNN architecture. For the next tasks, we will load the weights of the best epoch in terms of performance of our trained VGG16 in order to evaluate the model. We will use the test set for this purpose. Apply a threshold of 0.5 to obtain the class predictions and obtain the confusion matrix.

In [ ]:
import itertools

def plot_confusion_matrix(cm: np.array, 
                          classes: list, 
                          title: str = 'Confusion matrix', 
                          cmap: object = plt.cm.YlGn):
    """
    This function prints and plots the confusion matrix (cm). Normalization 
    can be applied by setting 'normalize=True'.
    
    Parameters: 
    ----------
    cm : np.array
        Confusion matrix.
    classes : list
        Data labels.
    normalize : bool, optional
        Apply or not normalization. The default is False.
    title : str, optional
        Title of the image. The default is 'Confusion matrix'.
    cmap : object, optional
        Color map. The default is 'plt.cm.YlGn'.
        
    Returns
    -------
    None
    
    """       
    
    plt.figure(figsize=(8, 8), dpi=144, facecolor='w', edgecolor='k')       
    
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    thresh = cm_norm.max() / 2.
    
    plt.imshow(cm_norm, interpolation='nearest', cmap=cmap, vmin=0, vmax=1)
    plt.title(title, fontsize=7)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, fontsize=7, wrap=True)
    plt.yticks(tick_marks, classes, fontsize=7, wrap=True)
  
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        cm = cm.astype('int')
        plt.text(j, i, (("%.2f (%d)" % (cm_norm[i, j], cm[i, j]))),
                     horizontalalignment="center", fontsize=7,
                     color="white" if cm_norm[i, j] > thresh else "black")
    
    plt.ylabel('True Label', fontsize=7)
    plt.xlabel('Predicted Label', fontsize=7)
    plt.tight_layout()

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the model
saved_model =  # Type your solution here

# Define test generator
test_gen = ImageDataGenerator(rescale=,) # Type your solution here
test_generator = test_gen.flow_from_dataframe(
    dataframe=, # Type your solution here
    x_col=, # Type your solution here
    y_col=None,
    class_mode=None,
    batch_size= 64,
    target_size=(224, 224),
    shuffle=False
)

In [ ]:
# Use the test set
predictions = saved_model.predict() # Type your solution here
Y_test = test_df['labels']

# Define threshold
threshold = # Type your solution here

# Threshold the predictions and apply argmax to get the class (axis=1)
class_pred = # Type your solution here

# Convert class_pred to categoricals with our encoder and inverse_transform()
class_pred_list = 

# Create confusion matrix
conf_mat = confusion_matrix(, , labels=df['labels'].unique()) # Type your solution here

# Obtain accuracy with argmax
acc_test = accuracy_score(, ) # Type your solution

print("Accuracy test:", acc_test)

plot_confusion_matrix(cm=conf_mat, classes=df['labels'].unique(),
                      title='Confusion matrix', cmap='Blues')

**Task 15**

Obtain a sklearn report (F1, precision and recall) of the test set.

**Q3**: What all these parameters stand for? Precision, Recall, F1-Score, Accuracy, Macro Average and Weighted Average.

In [ ]:
from sklearn.metrics import classification_report

# Generate a classification report
report = classification_report(, , target_names=df['labels'].unique()) # Type your solution here

print(report)

**Task 16**

Let's show some exmaples of the predictions!

Not too bad, isn't it?

In [ ]:
# Add the predicted labels to our test dataset
test_df['predicted_labels'] =  # Type your solution here

# Extract 9 random samples from test set
sample_test = # Type your solution here

# Plot the images with their predicted label
plt.figure(figsize=(12, 12))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['predicted_labels']
    img = load_img(filename, target_size=(256, 256))
    plt.subplot(3, 3, index+1)
    plt.imshow(img)
    plt.xlabel(category)
plt.tight_layout()
plt.show()

### QUESTIONS

**Q1** (Task 2):

**Q2** (Task 7):

**Q3** (Task 14):